## Embedding features

### Imports & Config

In [ ]:
import sys
import os
from pathlib import Path

# 自作ソースコード(src)が入ったDatasetのパスを通す
# 例: /kaggle/input/my-pasture-lib という名前でアップロードした場合
sys.path.append('/kaggle/input/my-pasture-lib')  # 適宜名前の変更が必要

import pandas as pd
import numpy as np
from src.config import CFG
from src.data.preprocessing import pivot_table, preprocess_image_path
from src.features.embedding import compute_embeddings, generate_semantic_features
from src.utils.logger import seeding

# --- Kaggle環境用に設定を上書き ---
CFG.BASE_PATH = Path("/kaggle/input/csiro-biomass")
CFG.TRAIN_CSV = CFG.BASE_PATH / "train.csv"
CFG.IMAGE_DIR = CFG.BASE_PATH / "train"
CFG.DEVICE = "cuda" # 特徴量抽出にはGPUが必須です

seeding(CFG.SEED)

### Read the Data

In [ ]:
train_raw = pd.read_csv(CFG.TRAIN_CSV)  # train.csvのパスも確認しだい,CFGのほうでも変更が必要
train_df = pivot_table(train_raw)
train_df = preprocess_image_path(train_df, CFG.IMAGE_DIR)

print(f"Total images: {len(train_df)}")

### Extract Features

In [ ]:
# 使用するモデルのパス（Kaggle Dataset上のパス）
siglip_path = "/kaggle/input/google-siglip-so400m-patch14-384"

# 1. AIベクトル（Embedding）の抽出
# 数時間かかる可能性があるため、進捗が見えるように tqdm が動きます
print("Extracting SigLIP embeddings...")
emb_df = compute_embeddings(train_df, model_path=siglip_path)

# 2. セマンティック特徴量（概念類似度）の生成
print("Generating semantic features...")
X_emb = emb_df.filter(like="emb").values
X_semantic = generate_semantic_features(X_emb, model_path=siglip_path)

### Save the Result

In [ ]:
# 保存ディレクトリの作成
os.makedirs("/kaggle/working/features", exist_ok=True)

# 1. 抽出したベクトルを保存 (NumPy形式は高速で軽量です)
np.save("/kaggle/working/features/train_siglip_embeddings.npy", X_emb)

# 2. セマンティック特徴量を保存
np.save("/kaggle/working/features/train_semantic_scores.npy", X_semantic)

# 3. 対応する image_path などの情報を保存（後で結合するため）
train_df[['image_path', 'Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']].to_csv(
    "/kaggle/working/features/train_meta.csv", index=False
)

print("Feature extraction completed and files saved!")